In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
# Added the currencies of the countries studied as well as all the currencies of EU and some major world currencies which may appear in the data
currencies = ['GBP', 'EUR', 'BGN', 'CZK', 'DKK', 'HUF', 'PLN', 'RON', 'SEK', 'TRY', 'RUB', 'ARS', 'ISK', 'CHF', 'JPY', 'NOK', 'PKR', 'AUD', 'QAR']

# The result of this will be a set of dataframes stored in a dict
df_dict = {}

# The currency we would like to convert everything to
base = "USD"

for currency in currencies:
    
    # Storing each currency in a separate dataframe
    df = pd.DataFrame(columns=["Date", currency])

    # API only allows to receive one year at the time
    # Generating a year for request
    for year in range(2016, 2024):
        start_date = f"{year}-01-01"
        if year != 2023:
            end_date = f"{year}-12-31"
        else:
            end_date = "2023-05-01"
    
        url = 'https://api.exchangerate.host/timeseries?base={0}&start_date={1}&end_date={2}&symbols={3}'.format(base,start_date,end_date,currency)
        
        response = requests.get(url)

        # We recieve json as response output
        # We need to convert it to dataframe
        data = response.json()

        rates=[]

        # For some currencies missing data may be present. It will not have an impact due to limited use of such currencies in the data
        for i,j in data["rates"].items():
            if currency in j.keys():
                rates.append([i, j[currency]])
            else:
                rates.append([i, np.nan])

        df_year = pd.DataFrame(rates, columns=["Date", currency])
        
        # 
        df = pd.concat([df, df_year])

    df_dict[currency] = df
        

In [10]:
# Adding all dict dataframes into a single wide dataframe
final_df = df_dict["GBP"]

for currency in currencies[1:]:
    final_df = pd.merge(final_df, df_dict[currency], on="Date", how="outer")

In [12]:
final_df.to_csv("exchange_rates.csv", index=False)